In [14]:
from numpy import mean
from numpy import std
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

In [15]:
# load data
train = pd.read_csv('../data/train_after_encode.csv')
test = pd.read_csv('../data/test_after_encode.csv')
protein_id = pd.read_csv('../data/protein_id.csv')
# define X,y
X_train = train.drop('y', axis=1)
y_train = train['y']
# create validation sets
#X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size = 0.3)

In [16]:
X_train = X_train.astype(np.float64)
y_train = y_train.astype(np.float64)

In [17]:
# logistic regression
lr = pd.read_pickle('../best models/LR.pickle')
rf = pd.read_pickle('../best models/random_forest.pickle')

In [19]:
# define the model
#model = BaggingClassifier(base_estimator=lr,verbose=1, n_jobs=-1)
model = BaggingClassifier(base_estimator=rf, verbose=1, n_jobs=-1)

In [20]:
# save bagging model
#pd.to_pickle(model, '../best models/LR_Bagging.pickle')
pd.to_pickle(model, '../best models/RF_Bagging.pickle')

In [6]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/Users/jasonyin/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/jasonyin/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/jasonyin/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/jasonyin/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated clas

Accuracy: 0.693 (0.042)


In [7]:
# model fit
model.fit(X_train,y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.6s remaining:    3.6s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s finished


BaggingClassifier(base_estimator=LogisticRegression(C=10, l1_ratio=0,
                                                    multi_class='ovr'),
                  n_jobs=-1, verbose=1)

The following is related to submission

In [8]:
# creating the result vector
y_pred_test = model.predict(test).astype(int)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


In [9]:
y_pred_test

array([ 1,  0,  0,  3,  5,  1,  0,  0,  5,  2,  4,  1,  0,  1,  0,  0,  0,
        1,  0,  0,  0,  3,  0,  0,  7,  0,  1,  0,  1,  0,  0,  0,  0,  0,
        0,  1,  2,  1, 10,  4,  0,  0,  0,  0,  0,  1,  1,  7,  0,  0,  6,
        0,  5,  0,  5,  0,  1,  4,  1,  0,  1,  0,  0,  3,  1,  0,  4,  5,
        0,  1,  0,  5,  0,  1,  0,  3,  1,  1,  0,  0,  3,  0,  1,  0,  0,
        0,  2,  0,  3,  2,  0,  0,  5,  0,  0,  3,  3,  3,  3,  1,  0,  0,
        1,  0,  3,  4,  4,  0,  1,  1,  6,  0,  3,  4,  0,  0,  2,  0,  0,
        3,  0,  0,  1,  0,  0,  0,  0,  2,  0,  1,  0,  1,  0,  5,  4,  3,
        1,  2,  1,  1,  0,  8,  5,  0,  2,  8,  5,  3,  2,  3,  1,  1,  1,
        2,  0,  1,  1,  0,  0,  0,  0,  2,  0,  1,  0,  0,  0,  7,  0,  0,
        3,  0,  5,  3,  6,  0,  1,  3,  0,  2,  3,  6,  0,  1,  0,  0,  0,
        1,  0,  1,  0,  1,  0,  1,  4,  3,  1,  0,  7,  2,  2,  0,  2,  0,
        2,  0,  3,  1,  4,  0,  0,  0,  2,  0,  6,  0,  5,  0,  0,  0,  1,
        7,  1,  1,  8,  7

In [10]:
# create result
res = {'key':protein_id['0'], 'label':y_pred_test}
res = pd.DataFrame(res)
# set id to be index
res = res.set_index('key')
# sort by index
res = res.sort_index()

In [11]:
res

,label
key,
P234062,2
P234081,0
P234086,2
P234087,0
P234094,0
...,...
P240380,0
P240407,2
P240440,1


In [12]:
res.to_csv('../result/submission_bagging.csv')